In [1]:
import torch
import os
import glob
import numpy as np

from diffusers import ReSDPipeline

from utils import eval_psnr_ssim_msssim, bytearray_to_bits
from watermarker import InvisibleWatermarker
from wmattacker import DiffWMAttacker, VAEWMAttacker, JPEGAttacker

/local/home/xuandongzhao/anaconda3/envs/wmatt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
wm_text = 'test'
device = 'cuda:0'
ori_path = 'examples/ori_imgs/'
output_path = 'examples/wm_imgs/'
print_width = 50

In [3]:
os.makedirs(output_path, exist_ok=True)
ori_img_paths = glob.glob(os.path.join(ori_path, '*.*'))
ori_img_paths = sorted([path for path in ori_img_paths if path.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif'))])
print(ori_img_paths)

['examples/ori_imgs/000000000711.png', 'examples/ori_imgs/000000000776.png', 'examples/ori_imgs/000000000885.png', 'examples/ori_imgs/000000000969.png', 'examples/ori_imgs/000000001089.png', 'examples/ori_imgs/000000001442.png']


In [4]:
wmarkers = {
    'DwtDct': InvisibleWatermarker(wm_text, 'dwtDct'),
    'DwtDctSvd': InvisibleWatermarker(wm_text, 'dwtDctSvd'),
    'RivaGAN': InvisibleWatermarker(wm_text, 'rivaGan'),
}

In [5]:
pipe = ReSDPipeline.from_pretrained("stabilityai/stable-diffusion-2-1", torch_dtype=torch.float16, revision="fp16")
pipe.set_progress_bar_config(disable=True)
pipe.to(device)
print('Finished loading model')

text_encoder/model.safetensors not found
Loading pipeline components...: 100%|██████████| 5/5 [00:02<00:00,  2.22it/s]


Finished loading model


In [6]:
attackers = {
    'diff_attacker_60': DiffWMAttacker(pipe, batch_size=5, noise_step=60, captions={}),
    'cheng2020-anchor_3': VAEWMAttacker('cheng2020-anchor', quality=3, metric='mse', device=device),
    'bmshj2018-factorized_3': VAEWMAttacker('bmshj2018-factorized', quality=3, metric='mse', device=device),
    'jpeg_attacker_50': JPEGAttacker(quality=50),
}

Diffuse attack initialized with noise step 60 and use prompt 0


In [7]:
def add_watermark(wmarker_name, wmarker):
    print('*' * print_width)
    print(f'Watermarking with {wmarker_name}')
    os.makedirs(os.path.join(output_path, wmarker_name + '/noatt'), exist_ok=True)
    for ori_img_path in ori_img_paths:
        img_name = os.path.basename(ori_img_path)
        wmarker.encode(ori_img_path, os.path.join(output_path, wmarker_name + '/noatt', img_name))

for wmarker_name, wmarker in wmarkers.items():
    add_watermark(wmarker_name, wmarker)
print('Finished watermarking')

**************************************************
Watermarking with DwtDct
**************************************************
Watermarking with DwtDctSvd
**************************************************
Watermarking with RivaGAN
Finished watermarking


In [8]:
for wmarker_name, wmarker in wmarkers.items():
    for attacker_name, attacker in attackers.items():
        print('*' * print_width)
        print(f'Attacking {wmarker_name} with {attacker_name}')
        wm_img_paths = []
        att_img_paths = []
        os.makedirs(os.path.join(output_path, wmarker_name, attacker_name), exist_ok=True)
        for ori_img_path in ori_img_paths:
            img_name = os.path.basename(ori_img_path)
            wm_img_paths.append(os.path.join(output_path, wmarker_name + '/noatt', img_name))
            att_img_paths.append(os.path.join(output_path, wmarker_name, attacker_name, img_name))
        attackers[attacker_name].attack(wm_img_paths, att_img_paths)
        
print('Finished attacking')

**************************************************
Attacking DwtDct with diff_attacker_60


4it [00:01,  3.06it/s]/share/edc/home/xuandongzhao/WatermarkAttacker/src/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py:429: FutureWarning: The decode_latents method is deprecated and will be removed in a future version. Please use VaeImageProcessor instead
  warnings.warn(
6it [00:02,  2.09it/s]


**************************************************
Attacking DwtDct with cheng2020-anchor_3


6it [00:00,  6.90it/s]


**************************************************
Attacking DwtDct with bmshj2018-factorized_3


...

**************************************************
Attacking RivaGAN with bmshj2018-factorized_3


6it [00:00,  7.52it/s]


**************************************************
Attacking RivaGAN with jpeg_attacker_50


6it [00:00, 69.73it/s]

Finished attacking


In [9]:
wm_results = {}
for wmarker_name, wmarker in wmarkers.items():
    print('*' * print_width)
    print(f'Watermark: {wmarker_name}')
    wm_successes = []
    wm_psnr_list = []
    wm_ssim_list = []
    wm_msssim_list = []
    for ori_img_path in ori_img_paths:
        img_name = os.path.basename(ori_img_path)
        wm_img_path = os.path.join(output_path, wmarker_name+'/noatt', img_name)
        wm_psnr, wm_ssim, wm_msssim = eval_psnr_ssim_msssim(ori_img_path, wm_img_path)
        wm_psnr_list.append(wm_psnr)
        wm_ssim_list.append(wm_ssim)
        wm_msssim_list.append(wm_msssim)
    wm_results[wmarker_name] = {}
    wm_results[wmarker_name]['wm_psnr'] = np.array(wm_psnr_list).mean()
    wm_results[wmarker_name]['wm_ssim'] = np.array(wm_ssim_list).mean()
    wm_results[wmarker_name]['wm_msssim'] = np.array(wm_msssim_list).mean()
    
print('Finished evaluating watermarking')

**************************************************
Watermark: DwtDct
**************************************************
Watermark: DwtDctSvd
**************************************************
Watermark: RivaGAN
Finished evaluating watermarking


In [10]:
wm_results

{'DwtDct': {'wm_psnr': 39.68427960034699,
  'wm_ssim': 0.9798382719357809,
  'wm_msssim': 0.9923724134763082},
 'DwtDctSvd': {'wm_psnr': 39.79678248735391,
  'wm_ssim': 0.9887041548887888,
  'wm_msssim': 0.9908552368481954},
 'RivaGAN': {'wm_psnr': 40.576834213869915,
  'wm_ssim': 0.9815376698970795,
  'wm_msssim': 0.9897827307383219}}

In [11]:
detect_wm_results = {}
for wmarker_name, wmarker in wmarkers.items():
    print('*' * print_width)
    print(f'Watermark: {wmarker_name}')
    bit_accs = []
    wm_successes = []
    for ori_img_path in ori_img_paths:
        img_name = os.path.basename(ori_img_path)
        wm_img_path = os.path.join(output_path, wmarker_name+'/noatt', img_name)
        wm_psnr, wm_ssim, wm_msssim = eval_psnr_ssim_msssim(ori_img_path, wm_img_path)
        wm_text = wmarkers[wmarker_name].decode(wm_img_path)
        try:
            if type(wm_text) == bytes:
                a = bytearray_to_bits('test'.encode('utf-8'))
                b = bytearray_to_bits(wm_text)
            elif type(wm_text) == str:
                a = bytearray_to_bits('test'.encode('utf-8'))
                b = bytearray_to_bits(wm_text.encode('utf-8'))
            bit_acc = (np.array(a) ==  np.array(b)).mean()
            bit_accs.append(bit_acc)
            if bit_acc > 24/32:
                wm_successes.append(img_name)
        except:
            print('#' * print_width)
            print(f'failed to decode {wm_text}', type(wm_text), len(wm_text))
            pass
    detect_wm_results[wmarker_name] = {}
    detect_wm_results[wmarker_name]['bit_acc'] = np.array(bit_accs).mean()
    
print('Finished evaluating watermarking')

**************************************************
Watermark: DwtDct
**************************************************
Watermark: DwtDctSvd
**************************************************
Watermark: RivaGAN
Finished evaluating watermarking


In [12]:
detect_wm_results

{'DwtDct': {'bit_acc': 0.8177083333333334},
 'DwtDctSvd': {'bit_acc': 1.0},
 'RivaGAN': {'bit_acc': 1.0}}